In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras
import keras.layers as L
from keras.datasets import fashion_mnist
from keras.applications import DenseNet121
from keras.applications import VGG16
import matplotlib.pyplot as plt

In [4]:
def show(img):
    plt.imshow(img)

In [5]:
def create_exemplar_set(size, images, trained_model):
    #'images' has images all of class=y
    exemplars=[]
    for i in images:
        exemplars.append(calc_feature_vector(i, trained_model, layer_index))
    chosen_exemplars=exemplars[:size]
    mean_exemplar=sum(exemplars)/len(images)
    return chosen_exemplars, mean_exemplar

 For multi-task learning, branching NN out
- one branch predicts the class of FashionMNIST object (classification)
- other branch predicts whether the object is a "top" or not (0/1 output)

In [6]:
def create_model(n_classes, input_dim, cl_weight, b_weight, lr):
    '''
        Creating categorical classification model
    '''
    
    inputs=L.Input((input_dim,input_dim,1))
    
    xc=L.Conv2D(64, kernel_size=3, padding='same', strides=1)(inputs)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.Conv2D(64, kernel_size=3, padding='same', strides=1)(xc)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.MaxPool2D(pool_size=2, strides=2)(xc)
    
    xc=L.Conv2D(32, kernel_size=3, padding='same', strides=1)(inputs)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.Conv2D(32, kernel_size=3, padding='same', strides=1)(xc)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.MaxPool2D(pool_size=2, strides=2)(xc)
    
    xc=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(xc)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(xc)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.MaxPool2D(pool_size=2, strides=2)(xc)
    
    xc=L.Flatten()(xc)
    xc=L.Dense(n_classes)(xc)
    outputc=L.Softmax(name='outputc')(xc)
    
    '''
        Creating binary classification model (a top/not a top)
    '''
    
    x=L.LeakyReLU(0.2)(inputs)
    x=L.Conv2D(32, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.MaxPool2D(pool_size=2, strides=2)(x)
    
    x=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.MaxPool2D(pool_size=2, strides=2)(x)
    
    x=L.Flatten()(x)
    outputb=L.Dense(1, activation='sigmoid', name='outputb')(x)
    #outputb=L.Activation('sigmoid', name="outputb")(x)

    '''
        Combining both models
    '''
    model=keras.Model(inputs=inputs, outputs=[outputc, outputb])
    losses={'outputc':'categorical_crossentropy', 'outputb':'binary_crossentropy'}
    loss_weights=[cl_weight, b_weight]
    model.compile(optimizer=keras.optimizers.Adam(lr), 
                      loss=losses,
                      loss_weights=loss_weights,
                      metrics=['accuracy'])
    
    return model

In [7]:
def train_model(model, n_epochs:int, x_train, y_trainc, y_trainb, validation_split):
    history=model.fit(x=x_train, 
                      y={'outputc':y_trainc, 'outputb': y_trainb}, 
                      epochs=n_epochs, 
                      validation_split=0.1)
    return model

In [8]:
def calc_feature_vector(image, model, layer_index):
    #extract feature vector from a layer of ALREADY-TRAINED model
    extractor=keras.Model(model.inputs(), [model.layers[layer_index].output])
    vector=extractor(image)
    #divide vector by norm
    return vector

In [9]:
def classify(X, mean_exemplar_sets):    
    image_vector=calc_feature_vector(X)
    #exemplar set closest to X feature map is chosen
    return None

In [10]:
def transform_outputs(y_train, y_test):
    labeldict={}
    labeldict[0]=1
    labeldict[1]=0
    labeldict[2]=1
    labeldict[3]=1
    labeldict[4]=1
    labeldict[5]=0
    labeldict[6]=1
    labeldict[7]=0
    labeldict[8]=0
    labeldict[9]=0
    
    y_trainb=[]
    y_testb=[]
    y_trainc=keras.utils.to_categorical(y_train)
    y_testc=keras.utils.to_categorical(y_test)
    
    for i in range(len(y_train)):
        y_trainb.append(labeldict[y_train[i]])
        
    for i in range(len(y_test)):
        y_testb.append(labeldict[y_test[i]])
    
    return y_trainc, y_testc, np.asarray(y_trainb), np.asarray(y_testb)

In [11]:
def main():
    (x_train, y_train), (x_test, y_test)=tf.keras.datasets.fashion_mnist.load_data()
    x_train=x_train.reshape((x_train.shape[0], 28,28,1))
    x_test=x_test.reshape((x_test.shape[0],28,28,1))
    y_trainc, y_testc, y_trainb, y_testb=transform_outputs(y_train, y_test)
    x_train=x_train/255.0
    x_test=x_test/255.0
    
    #np.savetxt("ytrain_transformed.csv", y_train, delimiter=',')
    #np.savetxt("ytest_transformed.csv", y_test, delimiter=',')
    
    #y_train=np.loadtxt(open("ytrain_transformed.csv", "rb"), delimiter=',')
    #y_test=np.loadtxt(open("ytest_transformed.csv", "rb"), delimiter=',')
    
    model=create_model(n_classes=10, input_dim=28, cl_weight=1.0, b_weight=1.5, lr=0.0005)
    
    train_model(model, 50, x_train, y_trainc, y_trainb, validation_split=0.1)
    
#-----------------------------------------------------------------------#

main()

Epoch 1/50
  35/1688 [..............................] - ETA: 2:37 - loss: 2.8216 - outputc_loss: 1.9565 - outputb_loss: 0.5767 - outputc_accuracy: 0.2732 - outputb_accuracy: 0.7420- ETA: 2:32 - loss: 3.2515 - outputc_loss: 2.2472 - outputb_loss: 0.6696 - outputc_accuracy: 0.1562 - 

KeyboardInterrupt: 

Colab output with weighted losses(1,1.5), lr=0.005:

Epoch 50/50
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0617 - outputc_loss: 0.0604 - outputb_loss: 8.6645e-04 - outputc_accuracy: 0.9773 - outputb_accuracy: 0.9997